# Safety First!

This is the Jupyter Notebook for my app, Safety First! The intention of this notebook is to create a Keras model that can be converted to CoreML (NaturalDisasterClassifier.mlmodel), which then can be used on xCode/Swift. This model builds an image classifier using a Convolutional Neural Network. The images are classified to be one of the three categories: Cyclone, Earthquake, or Flood.


# 1) Get Designated Inputs!

Gets all the Keras libraries needed.

In [131]:
from __future__ import print_function
from matplotlib import pyplot as plt

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
print("what's up")
import coremltools
# coremltools supports Keras version 2.0.6
print('keras version ', keras.__version__)

what's up
keras version  2.0.6


# 2) Extract images and put it into pickle files.

I access my image folders and create X and y pickle files (image and categories respsectively) to be used in my convolutional neural network.

In [132]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
import pickle

print("yo")


file_list = []
class_list = []

DATADIR = "data"

# All the categories you want your neural network to detect
CATEGORIES = ["Cyclone", "Earthquake", "Flood"]

# The size of the images that your neural network will use
IMG_SIZE = 50

# Checking or all images in the data folder
for category in CATEGORIES :
	path = os.path.join(DATADIR, category)
	for img in os.listdir(path):
		img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)

print("this is ", img_array)
training_data = []

def create_training_data():
	for category in CATEGORIES :
		path = os.path.join(DATADIR, category)
		class_num = CATEGORIES.index(category)
		for img in os.listdir(path):
			try :
				img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
				# new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
				new_array = cv2.resize(img_array, (50, 30))
				training_data.append([new_array, class_num])
			except Exception as e:
				pass

create_training_data()

random.shuffle(training_data)

X = [] #features
y = [] #labels

for features, label in training_data:
	X.append(features)
	y.append(label)

# X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X = np.array(X).reshape(-1, 50, 30, 1)

# Creating the files containing all the information about your model
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

yo
this is  [[223 223 224 ... 219 218 218]
 [222 223 224 ... 219 218 218]
 [222 223 224 ... 220 218 218]
 ...
 [100 100  98 ... 123 120 120]
 [ 95  94  93 ... 125 122 122]
 [ 86  89  93 ... 129 127 127]]


# 3) Convolutional Neural Network Construction

The layers of the CNN is seen in the output below.

In [133]:
# Opening the files about data
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0

# model = Sequential()
# model.add(Conv2D(32, (5, 5), input_shape=X.shape[1:], activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))
# model.add(Conv2D(128, (1, 1), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(3, activation='softmax'))

model = Sequential()
# 3 convolutional layers
model.add(Conv2D(32, (3, 3), input_shape = X.shape[1:], activation='relu'))
# model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 2 hidden layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(3, activation='softmax'))

# print(model.summary())
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 48, 28, 32)        320       
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 24, 14, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 22, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 11, 6, 64)         0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 9, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 4, 2, 64)          0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 4, 2, 64)          0         
__________

# 4) Train the data!

Fit the model, and it is trained using "callbacks_list". As seen in the output below, our image classifying accuracy is > 95%.

In [142]:
# model.compile(loss='sparse_categorical_crossentropy',
#                 optimizer='adam', metrics=['accuracy'])

checkpoint_path = "best_model.{epoch:02d}-{val_loss:.2f}.h5"
# checkpoint_dir = os.path.dirname(checkpoint_path)

callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]
print("yo")

model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

# model.fit(X, y, batch_size=32, epochs=40, callbacks=callbacks_list, validation_split=0.1)
# model.fit(X, y, batch_size=32, epochs=40, validation_split=0.1)
model.fit(X, y, batch_size=32, nb_epoch=40, verbose=1, callbacks=callbacks_list, validation_split=0.1)


print("done")

yo
Train on 405 samples, validate on 45 samples
Epoch 1/40
405/405 [==============================] - 5s - loss: 0.2145 - acc: 0.9136 - val_loss: 1.2300 - val_acc: 0.7111
Epoch 2/40
405/405 [==============================] - 0s - loss: 0.2142 - acc: 0.9210 - val_loss: 1.0506 - val_acc: 0.7111
Epoch 3/40
405/405 [==============================] - 1s - loss: 0.1775 - acc: 0.9506 - val_loss: 1.1021 - val_acc: 0.7111
Epoch 4/40
405/405 [==============================] - 1s - loss: 0.1138 - acc: 0.9654 - val_loss: 1.1965 - val_acc: 0.7111
Epoch 5/40
405/405 [==============================] - 0s - loss: 0.1069 - acc: 0.9704 - val_loss: 1.1740 - val_acc: 0.6889
Epoch 6/40
405/405 [==============================] - 0s - loss: 0.0999 - acc: 0.9753 - val_loss: 1.0509 - val_acc: 0.7111
Epoch 7/40
405/405 [==============================] - 0s - loss: 0.0528 - acc: 0.9926 - val_loss: 1.2072 - val_acc: 0.7333
Epoch 8/40
405/405 [==============================] - 0s - loss: 0.0583 - acc: 0.9852 - val

# 5) Create the .mlmodel to be used on xCode.

These lines convert our Keras model to be usable on CoreML, which is Apple's machine learning framework.

In [150]:
print('keras version ', keras.__version__)
from keras.layers import Softmax

output_labels = ["Cyclone", "Earthquake", "Flood"]
# output_labels = ["0", "1", "2"]

# For the first argument, use the filename of the newest .h5 file in the notebook folder.
coreml_mnist = coremltools.converters.keras.convert(
    'best_model.08-0.89.h5', input_names=['image'], output_names=['output'], 
    class_labels=output_labels, image_input_names='image')

keras version  2.0.6


ImportError: cannot import name 'Softmax' from 'keras.layers' (/Users/kevinmo/Documents/Natural Disaster App/actualenv/lib/python3.7/site-packages/keras/layers/__init__.py)